In [1]:
import pygame
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import random

from keras.layers import Dense,Input,Flatten
from keras.models import Model
from keras.models import model_from_json

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [2]:
def modelRoot(inputX,classes):
    X=Flatten()(inputX)
    X=Dense(8,activation='relu')(X)
    X=Dense(6,activation='relu')(X)
    X=Dense(classes,activation=None)(X)
    return X

In [3]:
inputShape= Input((4,1))
mRoot=modelRoot(inputShape,2)
model=Model(inputs=inputShape,outputs=mRoot)
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 1)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 14        
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________


In [4]:
json_file = open('modelFlappy.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("modelFlappyWeight.h5")

In [5]:
def nextAction(dataQ,explorationRate,willLearn):
    alives=dataQ.shape[0]
    ans=np.zeros(alives)
    bb=0
    for it in range(totalBirds):
        if birdAlive[it]==0:
            continue
        if random.random() > explorationRate or not willLearn:
            ans[bb]=np.argmax(dataQ[bb])
        else:
            rn=random.random()
            if rn>biasRn[it]:
                ans[bb]=0
            else :
                ans[bb]=1
        bb+=1
    return ans

In [6]:
iterations=1
willLearn=False
explorationRate=1.0
explorationDelta=1.0/iterations
lamda=0.95

for gen in range(iterations):

    #variables
    unit=4
    totalBirds=100
    winHeight=600
    winWidth=1024
    birdLen=20
    posX=np.zeros(totalBirds)
    posY=np.zeros(totalBirds)
    birdAlive=np.ones(totalBirds)
    score=np.zeros(totalBirds)
    rewards=np.zeros(totalBirds)
    velocity=np.zeros(totalBirds)
    maxVel=100
    timeUnit=0.1
    gravity=20.0
    totalPillars=6
    gapPillar=300
    pillX=np.zeros(totalPillars)
    pillY=np.zeros(totalPillars)
    biasRn=np.zeros(totalBirds)
    justNow=np.zeros(totalBirds)
    gapUp=200
    isJump=np.zeros(totalBirds)
    jumpDuration=np.zeros(totalBirds)
    jumpLen=10
    check=np.ones((totalBirds,totalPillars))

    temp=600
    for it in range(totalPillars):
        pillX[it]=temp
        temp+=gapPillar
        pillY[it]=winHeight-winHeight*0.1-random.random()*winHeight*(0.8-0.33)

    for it in range(totalBirds):
        jumpDuration[it]=-jumpLen-1
        posX[it]=100
        posY[it]=50+random.random()*(winHeight-100)
        biasRn[it]=random.random()

    #initialization
    pygame.init()
    pygame.font.init()
    win=pygame.display.set_mode((winWidth,winHeight))
    pygame.display.set_caption("Raw Flappy bird")
    CC=pygame.time.Clock()
    myfont = pygame.font.SysFont('Comic Sans MS', 15)

    #game loop
    done=False
    out=False
    while not done:

        #fixing
        CC.tick(50)
        #pygame.time.delay(20)
        win.fill((255,255,255))

        #events and keys
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                done=True
                out=True
        keys=pygame.key.get_pressed()
        if keys[pygame.K_SPACE]:
            for it in range(totalBirds):
                '''isJump[it]=1
                jumpDuration[it]=jumpLen'''
        
        ########################## reinforcement learning old state info
        alives=0
        oldStates=[]
        for it in range(totalBirds):
            if birdAlive[it]==0:
                continue
            alives+=1
            rewards[it]=0.1
            bY=posY[it]
            
            #############
            mn=10000
            for it2 in range(totalPillars):
                temp1=(pillX[it2]+birdLen)-(posX[it])
                if temp1>=0 and temp1<mn:
                    mn=temp1
                    dis=mn
                    pY=pillY[it2]-gapUp
          
            #############
            oldState=[]
            oldState.append(bY/winHeight)
            oldState.append(velocity[it]/100.0)
            oldState.append(dis/winWidth)
            oldState.append(pY/winHeight)
            oldState=np.asarray(oldState)
            oldStates.append(oldState)
        oldStates=np.asarray(oldStates)
        oldStates=oldStates.reshape(oldStates.shape[0],oldStates.shape[1],1)
        oldQs=model.predict(oldStates)
        ############  next Action
        action=nextAction(oldQs,explorationRate,willLearn)
        ##########################
        
        ##applying actions
        cur=0
        for it in range(totalBirds):
            if birdAlive[it]==1:
                if action[cur]==1:
                    '''isJump[it]=1
                    jumpDuration[it]=jumpLen'''
                    velocity[it]=-30
                cur+=1
        ###########################

        #updating states
        for it in range(totalBirds):
            '''if birdAlive[it]==0:
                continue
            if isJump[it]==1:
                if jumpDuration[it] >= -jumpLen:
                    if jumpDuration[it]>0:
                        direction=1
                    else :
                        direction=-1
                    posY[it]-=direction*(abs(jumpDuration[it])**1)
                    jumpDuration[it]-=1
                else :
                    isJump[it]=0
            else:
                posY[it]+=unit'''
            posY[it]+=velocity[it]*timeUnit+0.5*gravity*timeUnit*timeUnit
            velocity[it]+=gravity*timeUnit
            
        mx=0
        for it in range(totalPillars):
            pillX[it]-=unit
            mx=max(mx,pillX[it])
        for it in range(totalPillars):
            if pillX[it]<-50:
                pillX[it]=mx+gapPillar
                pillY[it]=winHeight-winHeight*0.1-random.random()*winHeight*(0.8-0.33)
                for it2 in range(totalBirds):
                    check[it2,it]=1
                break

        killer=0
        #intersection &&&& rewards
        for it in range(totalBirds):
            if birdAlive[it]==1:
                if posY[it]<50 or posY[it]+birdLen > winHeight-50:
                    birdAlive[it]=0
                    rewards[it]=-1
                    justNow[it]=1
                    killer+=1
                
        for it in range(totalBirds):
            if birdAlive[it]==1:
                for it2 in range(totalPillars):
                    if posX[it]+birdLen<pillX[it2]:
                        continue
                    if pillX[it2]+birdLen<posX[it]:
                        continue
                    downH=pillY[it2]
                    upH=pillY[it2]-gapUp
                    if posY[it]<upH or posY[it]+birdLen > downH :
                        birdAlive[it]=0
                        rewards[it]=-1
                        justNow[it]=1
                        killer+=1
                        break
                        
        #scores
        for it in range(totalBirds):
            if birdAlive[it]==0:
                continue
            for it2 in range(totalPillars):
                if pillX[it2]+birdLen<posX[it] and check[it,it2]:
                    score[it]+=1
                    check[it,it2]=0
                    rewards[it]=1
                    break
                        
        ########################## reinforcement learning new state info
        newStates=[]
        fuck=0
        for it in range(totalBirds):
            #print(str(it)+" ---al->"+str(birdAlive[it])+" jN->"+str(justNow[it]))
            if birdAlive[it]==0 and justNow[it]==0:
                continue
            fuck+=1
            bY=posY[it]
            
            #############
            mn=10000
            for it2 in range(totalPillars):
                temp1=(pillX[it2]+birdLen)-(posX[it])
                if temp1>=0 and temp1<mn:
                    mn=temp1
                    dis=mn
                    pY=pillY[it2]-gapUp
                    
            #############
            newState=[]
            newState.append(bY/winHeight)
            newState.append(velocity[it]/100.0)
            newState.append(dis/winWidth)
            newState.append(pY/winHeight)
            newStates.append(newState)
        newStates=np.asarray(newStates)
        newStates=newStates.reshape(newStates.shape[0],newStates.shape[1],1)
        newQs=model.predict(newStates)
        ##########################

        ########################## reinforcement learning train
        cur=0
        #print(oldQs.shape)
        #print(newQs.shape)
       #print(killer)
        #print(fuck)
        for it in range(totalBirds):
            if birdAlive[it]==0 and justNow[it]==0:
                continue
            okk=rewards[it] + lamda * np.amax(newQs[cur])
            oldQs[cur,int(action[cur])] = okk
            cur+=1
        if willLearn:
            model.fit(oldStates,oldQs,verbose=0)
        #############################################
            
        

        curAlives=totalBirds
        for it in range(totalBirds):
            justNow[it]=0
            if birdAlive[it]==0:
                curAlives-=1
        if curAlives==0:
            done=True


        #top and bottom boundaries
        pygame.draw.rect(win,(0,0,0),(0,0,winWidth,50))
        pygame.draw.rect(win,(0,0,0),(0,550,winWidth,50))

        #drawing bird ans pillars
        for it in range(totalBirds):
            if birdAlive[it]==0:
                continue
            pygame.draw.rect(win,(0,0,255),(posX[it],posY[it],birdLen,birdLen))
        for it in range(totalPillars):
            pygame.draw.rect(win,(0,0,0),(pillX[it],pillY[it],birdLen,winHeight-pillY[it]))
            pygame.draw.rect(win,(0,0,0),(pillX[it],0,birdLen,pillY[it]-gapUp))

        ##showing data
        highest=0
        cntAlive=0
        for it in range(totalBirds):
            if birdAlive[it]==1:
                cntAlive+=1
            highest=max(highest,score[it])
        
        text=myfont.render('Score: '+str(int(highest)),1,(255,255,255))
        win.blit(text,(winWidth-80,10))
        
        text=myfont.render('Alives: '+str(int(cntAlive)),1,(255,255,255))
        win.blit(text,(winWidth-80,560))
        
        text=myfont.render('Gen: '+str(int(gen)),1,(255,255,255))
        win.blit(text,(20,10))


        pygame.display.update()
        
    if explorationRate>0:
        explorationRate-=explorationDelta

    if out:
        break
pygame.quit()

In [7]:
'''# serialize model to JSON
model_json = model.to_json()
with open("modelFlappy.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelFlappyWeight.h5")
print("Saved model to disk")'''

'# serialize model to JSON\nmodel_json = model.to_json()\nwith open("modelFlappy.json", "w") as json_file:\n    json_file.write(model_json)\n# serialize weights to HDF5\nmodel.save_weights("modelFlappyWeight.h5")\nprint("Saved model to disk")'